In [1]:
import cv2
import numpy as np
from scipy.spatial import distance as d
from numpy import linalg as la
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline

In [9]:
imgarr = []
img = cv2.imread('yosemite1.jpg')
imgarr.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = cv2.imread('yosemite4.jpg')
imgarr.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = cv2.imread('yosemite3.jpg')
imgarr.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = cv2.imread('yosemite2.jpg')
imgarr.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [24]:
def img_match(image1, image2):
    sift = cv2.xfeatures2d.SIFT_create()
    src1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    src1 = np.uint8(src1)
    src2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
    src2 = np.uint8(src2)
    kp1 = sift.detect(src1, None)
    kp2 = sift.detect(src2, None)
    kp1, des1 = sift.compute(src1, kp1)
    kp2, des2 = sift.compute(src2, kp2)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.90*n.distance or n.distance < 0.90*m.distance:
            good.append((m.queryIdx, m.trainIdx))
            
    if(len(kp1) < len(kp2)):
        num_points = len(kp1)
    else:
        num_points = len(kp2)
    trans = np.zeros((3,3))
    if(len(good) >= 4):
        #print(good[0][0])
        img1_pts = []
        img2_pts = []
        for point in range(0,len(good)):
            img1_pts.append(kp1[good[point][0]].pt)
            img2_pts.append(kp2[good[point][1]].pt)
        #img1_pt1 = kp1[good[0][0].trainIdx].pt
        #img1_pt2 = kp1[good[1][0].trainIdx].pt
        #img1_pt3 = kp1[good[2][0].trainIdx].pt
        #img2_pt1 = kp2[good[0][1].trainIdx].pt
        #img2_pt2 = kp2[good[1][1].trainIdx].pt
        #img3_pt3 = kp2[good[2][1].trainIdx].pt
        #img1_arr = np.float32([img1_pt1,img2_pt2,img3_pt3])
        #img2_arr = np.float32([img2_pt1, img2_pt2, img3_pt3])
        #print(img1_arr)
        #trans = cv2.getAffineTransform(img1_arr,img2_arr)
        trans,status = cv2.findHomography(np.float32(img1_pts),np.float32(img2_pts), cv2.RANSAC, 4.0)
    return (len(good)/num_points,trans)

In [25]:
#print('Match score is', img_match(imgarr[1], imgarr[2]))
weights = np.zeros((len(imgarr), len(imgarr)))
for i in range(0,len(imgarr)):
    for j in range(0,len(imgarr)):
        if(i == j):
            weights[i][j] = -1
            continue
        value,func = img_match(imgarr[i], imgarr[j])
        weights[i][j] = value

In [26]:
height, width, channels = imgarr[0].shape
#(height*(len(imgarr)+2)
bigimage = np.zeros((height,width*(len(imgarr)+2),channels), np.uint8)
print(bigimage.shape)
print(imgarr[0].shape)

(480, 3840, 3)
(480, 640, 3)


In [27]:
def imgmerge(img_base,img_add,trans):
    warpheight = img_base.shape[0]
    warpwidth = img_base.shape[1]+img_add.shape[1]
    warpimg = np.zeros((warpheight,warpwidth,channels), np.uint8)
    warpimg[0:img_add.shape[0],0:img_add.shape[1]] = img_add
    added_width = trans[len(transformations)-1][0][2]
    for k in range(0,len(transformations)):
        warpimg = cv2.warpPerspective(warpimg,transformations[k], (warpwidth,warpheight))
    #warpimg = cv2.warpPerspective(img_add, trans[0], (warpwidth,warpheight))
    warpimg[0:img_base.shape[0], 0:img_base.shape[1]] = img_base
    result = warpimg[:,0:(int(img_base.shape[1]+added_width))]
    return result

In [28]:
transformations = []
done_images = []
result = imgarr[0].copy()
i = 0 #pointer to image of interest
done_images.append(i)
#for i in range(0,len(imgarr)):
#for i in range(0,1):
while(len(done_images) < len(imgarr)):
    bestmatch = -1
    bestmatch_val = 0
    for j in range(0,len(imgarr)):
        if(j in done_images):
            continue
        weight = weights[j][i]
        #if(weight == -1):
        #    weight = weights[i][j]
        if weight > bestmatch_val:
            bestmatch = j
            bestmatch_val = weight
    print("Best match for image " + str(i) + " is " + str(bestmatch))
    if(bestmatch != -1):
        value,func = img_match(imgarr[bestmatch], imgarr[i])
        transformations.append(func)
        #print(func)
        result = imgmerge(result,imgarr[bestmatch],transformations)
    done_images.append(i)
    i = bestmatch

Best match for image 0 is 3
Best match for image 3 is 2
Best match for image 2 is 1


In [29]:
#cv2.imshow('test',result)
cv2.imwrite('out.jpg',cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

True